In [15]:
import intrf
import markov
import numpy as np
import sys
import math
from random import uniform

In [38]:
# Returns the Probability matrix for our graphs
def get_PT(p=0.3, q=0.0):
    return [[  1-p,    p,    0.0,    0.0,    0.0,    0.0], # 0 
      [  0.0,  1-p,  p/2.0,    0.0,   p/2.0,   0.0], # 1
      [1/4.0,  0.0,  1/4.0,   1/4.0,  1/4.0,   0.0], # 2
      [    q,  0.0,    0.0,   0.9-q,    0.1,   0.0], # 3
      [  0.0,  0.0,    0.0,     0.0,  1/2.0, 1/2.0], # 4
      [  0.0,  0.0,    0.0,    1/4.0, 1/2.0, 1/4.0]  #5
      ]
#
# This class implements a Markov chain. Upon creation, a transition
# matrix and an initial state are given. The chain may be executed and
# it keeps track of the probability that at time t it is in state m.
#
class Markov:

    #
    # Initialization function
    #
    def __init__(self, PT, m_ini):
        self.PT = PT
        self.nstate = len(PT[0])
        self.state = m_ini
        self.steps = 1      # Number of steps executed so far (for the 
                            # computation of averages)
        self.count = self.nstate*[0]  # count of the times a state was visited
        self.count[self.state] = 1

    #
    # Given a state and the transition probability matrix PT, determine a
    # new state with the probabilities prescribed by the matrix. Returns
    # the value of the new state.
    #
    def _transition(self, m):
        p = self.PT[m]
        u = uniform(0.0, 1.0)
        a = 0
        for k in range(len(p)):
            a = a + p[k]
            if u < a:
                return k
        return len(p)-1

    #
    # Makes a step of the Markov chain: generates a new state, updates
    # the current state, the step counter and the state counter. 
    #
    # Returns the new state of the chain
    #
    def step(self):
        mnew = self._transition(self.state)
        self.state = mnew
        self.steps = self.steps+1
        self.count[self.state] = self.count[self.state] + 1
        return mnew

    #
    # Returns the probability of occupations of each one of the states
    # of the chain
    #
    def stats(self):
        s = sum(self.count)
        return [float(x)/float(s) for x in self.count]


#
# Runs a set of NC chains (with transition PT and initial state m_ini)
# for a total of T steps and at the end collects a collection
# statistics of the states in which the chains are.
#
def set_run(NC, PT, m_ini, T):
    chains = [Markov(PT, m_ini) for _ in range(NC)]
    state_no = len(PT[0])
    state = NC*[0]
    for t in range(T):
        for k in range(NC):
            state[k] = chains[k].step()

    count = state_no*[0.0]
    for k in range(NC):
        count[state[k]] = count[state[k]] + 1
    return [float(c)/float(NC) for c in count]


#
# Runs a set of NC chains (with transition PT and initial state m_ini)
# for a total of T steps and at the end returns probability of ever reaching each state
#
def set_run_h(NC, PT, m_ini, T):
    chains = [Markov(PT, m_ini) for _ in range(NC)]
    state_no = len(PT[0])
    state = np.zeros((NC, state_no))
    for t in range(T):
        for k in range(NC):
            state[k][chains[k].step()] = 1

    return state.mean(axis=0)

# Runs a set of NC chains (with transition PT and initial state m_ini)
# for a total of T steps and at the end returns the mean time to reach each state
#
def set_run_k(NC, PT, m_ini, T):
    chains = [Markov(PT, m_ini) for _ in range(NC)]
    state_no = len(PT[0])
    state = np.ones((NC, state_no)) * np.inf
    for t in range(T):
        for k in range(NC):
            c = chains[k].step()
            state[k][c] = t if state[k][c] == np.inf else state[k][c]

    return state.mean(axis=0)

In [39]:
starting_at = 0
q = 0.0
print(f"q = {q}")
print(set_run_h(500, get_PT(q=q), starting_at, 1000))
q = 0.1
print(f"q = {q}")
print(set_run_h(500, get_PT(q=q), starting_at, 1000))

q = 0.0
[0.754 1.    0.476 1.    1.    1.   ]
q = 0.1
[1. 1. 1. 1. 1. 1.]


In [40]:
starting_at = 0
q = 0.0
print(f"q = {q}")
print(set_run_k(500, get_PT(q=q), starting_at, 1000))
q = 0.1
print(f"q = {q}")
print(set_run_k(500, get_PT(q=q), starting_at, 1000))

q = 0.0
[   inf  2.02     inf 14.692  9.73  11.738]
q = 0.1
[ 7.888  2.336 43.79  15.464  9.686 11.686]
